### Building a Technical Analysis Chart with Python
* **Link:** https://medium.com/analytics-vidhya/building-a-technical-analysis-chart-with-python-17107b78b297
* **Method:** Building MACD, MA10,MA30 and RSI charts
  *	Used talib library to get MACD, MA10,MA30 and RSI. Very useful and easy. 
  *	Useful resource for pulling data and plotting it using common trading strategies such as MACD. 
  *	Good as a starting point. 
* **Libraries required:** Talib, mplfinance 
<h3>

We will import pandas for data storage and analysis.
Import datetime and dateutil for filtering data time range
Use talib to get measures sure as moving averages. 
Use mplfinance and matplotlib to plot stock price and indicators

In [11]:
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import talib
import matplotlib.pyplot as plt
from mplfinance.original_flavor import candlestick_ohlc
from matplotlib.pylab import date2num

Since we are using the link to get data on a stock, we need to define the to and from date we need the data from. 
To do that get todays date as a UTC timesstamp using datetime.  

In [12]:
today = datetime.today().strftime("%d/%m/%Y")
today = datetime.strptime(today + " +0000", "%d/%m/%Y %z")
to = int(today.timestamp())

Get ten years ago date in the same format too by using relativedelta where you can specify the exact number of years you want to go back from current date.   

In [13]:
ten_yr_ago = today-relativedelta(years=10)
fro = int(ten_yr_ago.timestamp())

Next, we create a function to use the yfinance url to get data for a set stock(ticker) and to and from dates. 
We then create a database using pandas and index it using the dates coloumn to add stock price information.

In [14]:
def get_price_hist(ticker):
    url = "https://query1.finance.yahoo.com/v7/finance/download/{ticker}?period1={fro}&period2={to}&interval=1d&events=history".format(ticker=ticker, fro=fro, to=to)
    data = pd.read_csv(url)
    data.index = data["Date"].apply(lambda x: pd.Timestamp(x))
    data.drop("Date", axis=1, inplace=True)

    return data

You can then call the function for any stock you want information on. In this case they used Netflix(Nflx)

In [15]:
nflx_df = get_price_hist("NFLX")
nflx_df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2011-03-14,29.032858,29.571428,28.402857,28.742857,28.742857,29913800
2011-03-15,28.925714,31.314285,28.857143,31.015715,31.015715,76736800
2011-03-16,31.575714,31.910000,30.321428,30.548571,30.548571,59555300
2011-03-17,30.991428,31.381430,30.407143,30.557142,30.557142,28709800
2011-03-18,30.508572,30.634287,29.617144,29.914286,29.914286,38354400
...,...,...,...,...,...,...
2021-03-05,511.980011,517.760010,498.790009,516.390015,516.390015,5068700
2021-03-08,514.460022,518.840027,492.850006,493.329987,493.329987,3979000
2021-03-09,507.309998,513.109985,503.820007,506.440002,506.440002,3462000


We then create a function to get indicators such as MACD, simple moving average and RSI. 
* When using Talib, it automatically gets the MACD, its signal(the 9day exponetial moving average) and a coloumn which helps plot a histogram showing the difference between MACD and its signal. 
* Using talib, you can calculate any indicators by calling their functions. such as talib.MA()

In [16]:
def get_indicators(data):
    data["macd"], data["macd_signal"], data["macd_hist"] = talib.MACD(data['Close'])
    
    # Get MA10 and MA30
    data["ma10"] = talib.MA(data["Close"], timeperiod=10)
    data["ma30"] = talib.MA(data["Close"], timeperiod=30)
    
    # Get RSI
    data["rsi"] = talib.RSI(data["Close"])
    return data

nflx_df2 = get_indicators(nflx_df)
nflx_df2

,Open,High,Low,Close,Adj Close,Volume,macd,macd_signal,macd_hist,ma10,ma30,rsi
Date,,,,,,,,,,,,
2011-03-14,29.032858,29.571428,28.402857,28.742857,28.742857,29913800,NaN,NaN,NaN,NaN,NaN,NaN
2011-03-15,28.925714,31.314285,28.857143,31.015715,31.015715,76736800,NaN,NaN,NaN,NaN,NaN,NaN
2011-03-16,31.575714,31.910000,30.321428,30.548571,30.548571,59555300,NaN,NaN,NaN,NaN,NaN,NaN
2011-03-17,30.991428,31.381430,30.407143,30.557142,30.557142,28709800,NaN,NaN,NaN,NaN,NaN,NaN
2011-03-18,30.508572,30.634287,29.617144,29.914286,29.914286,38354400,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-05,511.980011,517.760010,498.790009,516.390015,516.390015,5068700,-3.752082,1.119515,-4.871597,536.573007,545.174336,40.673520
2021-03-08,514.460022,518.840027,492.850006,493.329987,493.329987,3979000,-6.787180,-0.461824,-6.325356,532.528003,542.779669,34.062707
2021-03-09,507.309998,513.109985,503.820007,506.440002,506.440002,3462000,-8.041945,-1.977848,-6.064097,528.557001,541.101668,40.030348


Once we have our historical data on the Netflix stock and all the indicators, we can create a function to plot them using matplotlib and mpl finance. 

In [ ]:
def plot_chart(data, n, ticker):
    
    # Filter number of observations to plot
    data = data.iloc[-n:]
    
    # Create figure and set axes for subplots
    fig = plt.figure()
    fig.set_size_inches((20, 16))
    ax_candle = fig.add_axes((0, 0.72, 1, 0.32))
    ax_macd = fig.add_axes((0, 0.48, 1, 0.2), sharex=ax_candle)
    ax_rsi = fig.add_axes((0, 0.24, 1, 0.2), sharex=ax_candle)
    ax_vol = fig.add_axes((0, 0, 1, 0.2), sharex=ax_candle)
    
    # Format x-axis ticks as dates
    ax_candle.xaxis_date()
    
    # Get nested list of date, open, high, low and close prices
    ohlc = []
    for date, row in data.iterrows():
        openp, highp, lowp, closep = row[:4]
        ohlc.append([date2num(date), openp, highp, lowp, closep])
 
    # Plot candlestick chart
    ax_candle.plot(data.index, data["ma10"], label="MA10")
    ax_candle.plot(data.index, data["ma30"], label="MA30")
    candlestick_ohlc(ax_candle, ohlc, colorup="g", colordown="r", width=0.8)
    ax_candle.legend()
    
    # Plot MACD
    ax_macd.plot(data.index, data["macd"], label="macd")
    ax_macd.bar(data.index, data["macd_hist"] * 3, label="hist")
    ax_macd.plot(data.index, data["macd_signal"], label="signal")
    ax_macd.legend()
    
    # Plot RSI
    # Above 70% = overbought, below 30% = oversold
    ax_rsi.set_ylabel("(%)")
    ax_rsi.plot(data.index, [70] * len(data.index), label="overbought")
    ax_rsi.plot(data.index, [30] * len(data.index), label="oversold")
    ax_rsi.plot(data.index, data["rsi"], label="rsi")
    ax_rsi.legend()
    
    # Show volume in millions
    ax_vol.bar(data.index, data["Volume"] / 1000000)
    ax_vol.set_ylabel("(Million)")
   
    # Save the chart as PNG
    fig.savefig("charts/" + ticker + ".png", bbox_inches="tight")
    
    plt.show()

Call the function with desired stock

In [ ]:
plot_chart(nflx_df2, 180, "NFLX")